In [ ]:
!pip install pandas numpy scikit-learn nltk
!pip install transformers torch
!pip install fastapi uvicorn
!pip install langdetect
!pip install sinling  # For Sinhala text processing
!pip install indic-nlp-library  # For Tamil text processing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=94670e28e0321d88c9850578a63152113783508fbfd64ea10cf36d1e38620a11
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 14.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
import pickle
import warnings
warnings.filterwarnings('ignore')

# Language detection
from langdetect import detect
import nltk
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)

print("✅ All packages installed successfully!")

✅ All packages installed successfully!


In [ ]:
try:
    df = pd.read_excel('/mnt/data/cleaned_phishing_data.xlsx')
    print("✅ Dataset loaded successfully!")
    print(f"Dataset shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print("\nFirst few rows:")
    print(df.head())
except Exception as e:
    print(f"❌ Error loading dataset: {e}")
    print("Please upload your Excel file to Colab first")

✅ Dataset loaded successfully!
Dataset shape: (300, 2)
Columns: ['Email Text', 'Email Type']

First few rows:
                                          Email Text Email Type
0  Dear Jordan, your subscription has been succes...       safe
1  ආදරණීය Jordan, ඔබගේ දායකත්වය සාර්ථකව නැවත යාවත...       safe
2  அன்புள்ள Jordan, உங்கள் சந்தா வெற்றிகரமாக புது...       safe
3  Dear Casey, thank you for your purchase. Your ...       safe
4  ආදරණීය Casey, ඔබගේ මිලදී ගැනීම සඳහා ස්තුතියි. ...       safe


In [ ]:
class MultilingualTextPreprocessor:
    def __init__(self):
        self.english_stopwords = set(['the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at',
                                     'to', 'for', 'of', 'with', 'by', 'is', 'are', 'was', 'were'])

        # Sinhala stopwords (common words to remove)
        self.sinhala_stopwords = set(['මම', 'ඔබ', 'ඔහු', 'ඇය', 'අපි', 'ඔබට', 'ඔබගේ', 'මගේ',
                                     'එම', 'මෙම', 'සහ', 'නමුත්', 'හෝ', 'ද', 'ට', 'ගේ', 'යන'])

        # Tamil stopwords (common words to remove)
        self.tamil_stopwords = set(['நான்', 'நீ', 'அவன்', 'அவள்', 'நாம்', 'உங்கள்', 'என்',
                                   'அந்த', 'இந்த', 'மற்றும்', 'ஆனால்', 'அல்லது', 'உள்ள', 'க்கு'])

    def detect_language(self, text):
        """Detect language of text"""
        try:
            # Check for Sinhala characters
            if re.search(r'[\u0D80-\u0DFF]', text):
                return 'sinhala'
            # Check for Tamil characters
            elif re.search(r'[\u0B80-\u0BFF]', text):
                return 'tamil'
            # Default to English
            else:
                return 'english'
        except:
            return 'english'

    def clean_text(self, text, language):
        """Clean text based on language"""
        if pd.isna(text):
            return ""

        text = str(text).strip()

        if language == 'english':
            # English preprocessing
            text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' URL ', text)
            text = re.sub(r'www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' URL ', text)
            text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', ' EMAIL ', text)
            text = re.sub(r'\b\d+\b', ' NUMBER ', text)
            text = re.sub(r'[^\w\s]', ' ', text)
            text = re.sub(r'\s+', ' ', text)
            text = text.lower()

        elif language == 'sinhala':
            # Sinhala preprocessing
            text = re.sub(r'http[s]?://\S+', ' URL ', text)
            text = re.sub(r'www\.\S+', ' URL ', text)
            text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', ' EMAIL ', text)
            text = re.sub(r'[0-9]+', ' NUMBER ', text)
            # Keep Sinhala characters, spaces, and basic punctuation
            text = re.sub(r'[^\u0D80-\u0DFF\s]', ' ', text)
            text = re.sub(r'\s+', ' ', text)

        elif language == 'tamil':
            # Tamil preprocessing
            text = re.sub(r'http[s]?://\S+', ' URL ', text)
            text = re.sub(r'www\.\S+', ' URL ', text)
            text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', ' EMAIL ', text)
            text = re.sub(r'[0-9]+', ' NUMBER ', text)
            # Keep Tamil characters, spaces, and basic punctuation
            text = re.sub(r'[^\u0B80-\u0BFF\s]', ' ', text)
            text = re.sub(r'\s+', ' ', text)

        return text.strip()

    def remove_stopwords(self, text, language):
        """Remove stopwords based on language"""
        words = text.split()

        if language == 'english':
            words = [word for word in words if word.lower() not in self.english_stopwords]
        elif language == 'sinhala':
            words = [word for word in words if word not in self.sinhala_stopwords]
        elif language == 'tamil':
            words = [word for word in words if word not in self.tamil_stopwords]

        return ' '.join(words)

    def preprocess(self, text):
        """Complete preprocessing pipeline"""
        if pd.isna(text) or text == "":
            return "", "unknown"

        # Detect language
        language = self.detect_language(text)

        # Clean text
        cleaned_text = self.clean_text(text, language)

        # Remove stopwords
        final_text = self.remove_stopwords(cleaned_text, language)

        return final_text, language

# Initialize preprocessor
preprocessor = MultilingualTextPreprocessor()

In [ ]:
def prepare_dataset(df):
    """Prepare the dataset for training"""

    # Print original columns to debug
    print(f"Original columns: {list(df.columns)}")
    print(f"Original shape: {df.shape}")

    # Rename columns for consistency (adjust based on your dataset)
    if 'Email Text' in df.columns:
        df = df.rename(columns={'Email Text': 'text'})
    if 'Email Type' in df.columns:
        df = df.rename(columns={'Email Type': 'label'})

    # Print unique labels to see what we have
    print(f"Unique labels before processing: {df['label'].unique()}")

    # Clean the dataset
    df = df.dropna(subset=['text', 'label'])

    # Standardize labels - handle both 'safe/unsafe' and 'legitimate/scam'
    df['label'] = df['label'].astype(str).str.lower().str.strip()

    # Map different label formats to standard format
    label_mapping = {
        'safe': 'legitimate',
        'unsafe': 'scam',
        'legitimate': 'legitimate',
        'scam': 'scam',
        '0': 'legitimate',  # In case labels are numeric
        '1': 'scam',
        0: 'legitimate',
        1: 'scam'
    }

    df['label'] = df['label'].map(label_mapping)

    # Keep only valid labels
    df = df[df['label'].isin(['legitimate', 'scam'])]

    print(f"Labels after mapping: {df['label'].value_counts()}")

    # Preprocess text and detect language
    processed_data = []

    for idx, row in df.iterrows():
        text = row['text']
        label = row['label']

        if pd.isna(text) or str(text).strip() == '':
            continue

        # Preprocess text
        processed_text, detected_language = preprocessor.preprocess(str(text))

        if processed_text.strip():  # Only include non-empty texts
            processed_data.append({
                'original_text': text,
                'processed_text': processed_text,
                'language': detected_language,
                'label': label
            })

    processed_df = pd.DataFrame(processed_data)

    print(f"✅ Dataset processed successfully!")
    print(f"Total samples: {len(processed_df)}")
    print(f"Language distribution:")
    print(processed_df['language'].value_counts())
    print(f"Label distribution:")
    print(processed_df['label'].value_counts())

    return processed_df

# Process your dataset
processed_df = prepare_dataset(df)

Original columns: ['Email Text', 'Email Type']
Original shape: (300, 2)
Unique labels before processing: ['safe' 'unsafe']
Labels after mapping: label
legitimate    231
scam           69
Name: count, dtype: int64
✅ Dataset processed successfully!
Total samples: 300
Language distribution:
language
english    100
sinhala    100
tamil      100
Name: count, dtype: int64
Label distribution:
label
legitimate    231
scam           69
Name: count, dtype: int64


In [ ]:
class MultilingualScamDetector:
    def __init__(self):
        self.models = {}
        self.vectorizers = {}
        self.preprocessor = MultilingualTextPreprocessor()

    def train_language_model(self, texts, labels, language):
        """Train a model for specific language"""
        print(f"Training model for {language}...")

        # Create TF-IDF vectorizer with language-specific parameters
        if language == 'english':
            vectorizer = TfidfVectorizer(
                max_features=5000,
                ngram_range=(1, 2),
                min_df=2,
                max_df=0.8
            )
        else:  # Sinhala and Tamil
            vectorizer = TfidfVectorizer(
                max_features=3000,
                ngram_range=(1, 3),
                min_df=1,
                max_df=0.9,
                analyzer='char'  # Character-level analysis
            )

        # Vectorize texts
        X = vectorizer.fit_transform(texts)

        # Train model
        model = LogisticRegression(
            random_state=42,
            max_iter=1000,
            class_weight='balanced'  # Handle imbalanced data
        )
        model.fit(X, labels)

        # Store model and vectorizer
        self.models[language] = model
        self.vectorizers[language] = vectorizer

        # Get feature names for keyword extraction
        feature_names = vectorizer.get_feature_names_out()

        # Print training results
        y_pred = model.predict(X)
        accuracy = accuracy_score(labels, y_pred)
        print(f"✅ {language.title()} model trained - Accuracy: {accuracy:.3f}")

        return accuracy

    def train_all_models(self, processed_df):
        """Train models for all languages"""
        results = {}

        for language in processed_df['language'].unique():
            if language == 'unknown':
                continue

            # Filter data for this language
            lang_data = processed_df[processed_df['language'] == language]

            if len(lang_data) < 10:  # Skip if too few samples
                print(f"⚠️ Skipping {language} - insufficient data ({len(lang_data)} samples)")
                continue

            # Get texts and labels
            texts = lang_data['processed_text'].tolist()
            labels = lang_data['label'].tolist()

            # Train model
            accuracy = self.train_language_model(texts, labels, language)
            results[language] = accuracy

        print(f"\n🎉 Training completed for all languages!")
        return results

    def predict(self, text):
        """Predict if text is scam or legitimate"""
        # Preprocess text
        processed_text, language = self.preprocessor.preprocess(text)

        if not processed_text.strip():
            return {
                'text': text,
                'language': 'unknown',
                'classification': 'unknown',
                'risk_score': 0,
                'suspicious_terms': [],
                'explanation': 'Unable to process text'
            }

        # Check if we have a model for this language
        if language not in self.models:
            # Try English model as fallback
            if 'english' in self.models:
                language = 'english'
                processed_text, _ = self.preprocessor.preprocess(text)
            else:
                return {
                    'text': text,
                    'language': language,
                    'classification': 'unknown',
                    'risk_score': 0,
                    'suspicious_terms': [],
                    'explanation': f'No model available for {language}'
                }

        # Get model and vectorizer
        model = self.models[language]
        vectorizer = self.vectorizers[language]

        # Vectorize text
        X = vectorizer.transform([processed_text])

        # Predict
        prediction = model.predict(X)[0]
        probability = model.predict_proba(X)[0]

        # Get risk score (probability of being scam)
        scam_prob = probability[1] if model.classes_[1] == 'scam' else probability[0]
        risk_score = int(scam_prob * 100)

        # Get suspicious terms (top features with highest coefficients)
        suspicious_terms = self.get_suspicious_terms(vectorizer, model, processed_text)

        # Generate explanation
        explanation = self.generate_explanation(prediction, risk_score, suspicious_terms, language)

        return {
            'text': text,
            'language': language,
            'classification': prediction,
            'risk_score': risk_score,
            'suspicious_terms': suspicious_terms,
            'explanation': explanation
        }

    def get_suspicious_terms(self, vectorizer, model, text, top_n=3):
        """Extract suspicious terms from text"""
        try:
            # Get feature names
            feature_names = vectorizer.get_feature_names_out()

            # Get model coefficients
            if hasattr(model, 'coef_'):
                coefficients = model.coef_[0]
            else:
                return []

            # Vectorize the text
            X = vectorizer.transform([text])

            # Get non-zero features (words present in text)
            feature_indices = X.nonzero()[1]

            # Get suspicious terms (positive coefficients for scam class)
            suspicious_indices = []
            for idx in feature_indices:
                if coefficients[idx] > 0:  # Positive coefficient indicates scam
                    suspicious_indices.append((idx, coefficients[idx]))

            # Sort by coefficient value and get top terms
            suspicious_indices.sort(key=lambda x: x[1], reverse=True)

            suspicious_terms = []
            for idx, coef in suspicious_indices[:top_n]:
                term = feature_names[idx]
                suspicious_terms.append(term)

            return suspicious_terms

        except Exception as e:
            print(f"Error extracting suspicious terms: {e}")
            return []

    def generate_explanation(self, prediction, risk_score, suspicious_terms, language):
        """Generate explanation in appropriate language"""
        explanations = {
            'english': {
                'scam': f"This message appears to be a SCAM (Risk: {risk_score}%). Suspicious elements detected.",
                'legitimate': f"This message appears to be LEGITIMATE (Risk: {risk_score}%). No major red flags detected."
            },
            'sinhala': {
                'scam': f"මෙම පණිවිඩය වංචනික (අවදානම: {risk_score}%) බව පෙනේ. සැක සහිත අංග හමුවිය.",
                'legitimate': f"මෙම පණිවිඩය නීත්‍යානුකූල (අවදානම: {risk_score}%) බව පෙනේ. විශාල රතු කොඩි හමු නොවිය."
            },
            'tamil': {
                'scam': f"இந்த செய்தி மோசடியாக (ஆபத்து: {risk_score}%) தெரிகிறது. சந்தேகத்திற்குரிய கூறுகள் கண்டறியப்பட்டன.",
                'legitimate': f"இந்த செய்தி சட்டபூர்வமானதாக (ஆபத்து: {risk_score}%) தெரிகிறது. பெரிய சிவப்பு கொடிகள் எதுவும் கண்டறியப்படவில்லை."
            }
        }

        base_explanation = explanations.get(language, explanations['english'])[prediction]

        if suspicious_terms:
            if language == 'english':
                base_explanation += f" Suspicious terms: {', '.join(suspicious_terms[:3])}"
            elif language == 'sinhala':
                base_explanation += f" සැක සහිත වචන: {', '.join(suspicious_terms[:3])}"
            elif language == 'tamil':
                base_explanation += f" சந்தேகத்திற்குரிய சொற்கள்: {', '.join(suspicious_terms[:3])}"

        return base_explanation

# Initialize and train the detector
detector = MultilingualScamDetector()
training_results = detector.train_all_models(processed_df)

Training model for english...
✅ English model trained - Accuracy: 1.000
Training model for sinhala...
✅ Sinhala model trained - Accuracy: 1.000
Training model for tamil...
✅ Tamil model trained - Accuracy: 1.000

🎉 Training completed for all languages!


In [ ]:
test_messages = [
    "CONGRAT UR NUMBER IS SELECTED AS WINER OF 189,000 POUND IN THE U.N FUND RELIEF,TO GET UR WINING,EMAIL US UR,NAME,ADDRESS,NUM TO EMAIL:nationalun756@yahoo.com",
    "සෑම නිවසකටම රුපියල් 50,000 ක ආධාර මුදලක් ලබාදීමට ජනාධිපතිවරයා අනුමැතිය ලබාදී තිබෙනවා.",

]

print("🧪 Testing the model:\n")

for i, message in enumerate(test_messages, 1):
    print(f"--- Test {i} ---")
    result = detector.predict(message)

    print(f"Message: {result['text']}")
    print(f"Language: {result['language']}")
    print(f"Risk Score: {result['risk_score']}%")
    print(f"Classification: {result['classification']}")
    print(f"Suspicious Terms: {result['suspicious_terms']}")
    print(f"Explanation: {result['explanation']}")
    print()

🧪 Testing the model:

--- Test 1 ---
Message: CONGRAT UR NUMBER IS SELECTED AS WINER OF 189,000 POUND IN THE U.N FUND RELIEF,TO GET UR WINING,EMAIL US UR,NAME,ADDRESS,NUM TO EMAIL:nationalun756@yahoo.com
Language: english
Risk Score: 44%
Classification: legitimate
Suspicious Terms: ['number']
Explanation: This message appears to be LEGITIMATE (Risk: 44%). No major red flags detected. Suspicious terms: number

--- Test 2 ---
Message: සෑම නිවසකටම රුපියල් 50,000 ක ආධාර මුදලක් ලබාදීමට ජනාධිපතිවරයා අනුමැතිය ලබාදී තිබෙනවා.
Language: sinhala
Risk Score: 49%
Classification: legitimate
Suspicious Terms: ['ක්', 'මට', 'මට ']
Explanation: මෙම පණිවිඩය නීත්‍යානුකූල (අවදානම: 49%) බව පෙනේ. විශාල රතු කොඩි හමු නොවිය. සැක සහිත වචන: ක්, මට, මට 



In [ ]:
test_messages = [
    "CONGRAT UR NUMBER IS SELECTED AS WINER OF 189,000 POUND IN THE U.N FUND RELIEF,TO GET UR WINING,EMAIL US UR,NAME,ADDRESS,NUM TO EMAIL:nationalun756@yahoo.com",
    "සෑම නිවසකටම රුපියල් 50,000 ක ආධාර මුදලක් ලබාදීමට ජනාධිපතිවරයා අනුමැතිය ලබාදී තිබෙනවා.",
    "CONGRATS,UR NUMBER IS WINNER OF 189,000 POUND IN THE U.N FUND RELIEF PROGRAM,TO CLAIM UR WINNING,EMAIL US UR FULL NAME,ADDRES,NUM TO EMAIL:relieffund2@yahoo.com",
    "You'll receive an otp code to redeem your cash prize of 100,00 rupees, click the link below and enter the otp you received in the website to get your cash prize",

]

print("🧪 Testing the model:\n")

for i, message in enumerate(test_messages, 1):
    print(f"--- Test {i} ---")
    result = detector.predict(message)

    print(f"Message: {result['text']}")
    print(f"Language: {result['language']}")
    print(f"Risk Score: {result['risk_score']}%")
    print(f"Classification: {result['classification']}")
    print(f"Suspicious Terms: {result['suspicious_terms']}")
    print(f"Explanation: {result['explanation']}")
    print()

🧪 Testing the model:

--- Test 1 ---
Message: CONGRAT UR NUMBER IS SELECTED AS WINER OF 189,000 POUND IN THE U.N FUND RELIEF,TO GET UR WINING,EMAIL US UR,NAME,ADDRESS,NUM TO EMAIL:nationalun756@yahoo.com
Language: english
Risk Score: 44%
Classification: legitimate
Suspicious Terms: ['number']
Explanation: This message appears to be LEGITIMATE (Risk: 44%). No major red flags detected. Suspicious terms: number

--- Test 2 ---
Message: සෑම නිවසකටම රුපියල් 50,000 ක ආධාර මුදලක් ලබාදීමට ජනාධිපතිවරයා අනුමැතිය ලබාදී තිබෙනවා.
Language: sinhala
Risk Score: 49%
Classification: legitimate
Suspicious Terms: ['ක්', 'මට', 'මට ']
Explanation: මෙම පණිවිඩය නීත්‍යානුකූල (අවදානම: 49%) බව පෙනේ. විශාල රතු කොඩි හමු නොවිය. සැක සහිත වචන: ක්, මට, මට 

--- Test 3 ---
Message: CONGRATS,UR NUMBER IS WINNER OF 189,000 POUND IN THE U.N FUND RELIEF PROGRAM,TO CLAIM UR WINNING,EMAIL US UR FULL NAME,ADDRES,NUM TO EMAIL:relieffund2@yahoo.com
Language: english
Risk Score: 47%
Classification: legitimate
Suspicious Terms: 

In [ ]:
# Save all models and vectorizers
model_data = {
    'models': detector.models,
    'vectorizers': detector.vectorizers,
    'preprocessor': detector.preprocessor
}

with open('clicksafe_multilingual_detector.pkl', 'wb') as f:
    pickle.dump(model_data, f)

print("✅ Model saved as 'clicksafe_multilingual_detector.pkl'")

✅ Model saved as 'clicksafe_multilingual_detector.pkl'


In [ ]:
# Download the .pkl file to your computer
from google.colab import files

# The file should already be created by your original code
# If not, run this:
# save_model_to_pkl(detector)

# Download the file
files.download('clicksafe_multilingual_detector.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>